### 数据说明
训练集包括文件：`News_info_train/Pic_info_train/News_pic_label_train`和`News_info_unlabel/Pic_info_unlabel`

验证集包括文件：`News_info_validate/Pic_info_validate/News_pic_validate`


说明：

1、 初赛的训练集数据在25万网页级别，复赛的测试数据集在1万网页级别；

 

2、 初赛数据在2018年3月19日中午12:00在比赛官网公开发布，所有测试数据在决赛后公开发布；

 

3、 训练集用于模型的学习，验证集用于在线实时评估算法效果；

 

4、 数据包含七个文件，格式如下：

 

4-1、News_info_train 数据集文本文件中的每一行表示：新闻ID，新闻文本，新闻配图ID列表（以英文分号间隔，没有配图时为NULL），上述三个字段以Tab间隔。

 

4-2、Pic_info_train 数据集中每一个文件是一张配图，配图的名称为配图ID。

 

4-3、News_pic_label_train 文件的每一行表示：新闻ID，新闻标注类别，有营销意图的配图ID列表（以英文分号间隔，没有时为NULL），有营销意图的文本片段（以Tab间隔），上述四个字段以Tab间隔。其中新闻标注类别的取值为{0, 1, 2}中的某一项，0: 无营销意图，1: 部分文本或配图有营销意图，2: 整篇新闻都有营销意图。

 

4-4、News_info_unlabel 数据集为未标注数据集，格式和News_info_train一样。

 

4-5、Pic_info_unlabel 数据集为未标注数据集，格式和Pic_info_train一样。

 

4-6、News_info_validate 是用于在线实时评测的新闻数据集，格式和News_info_train一样。

 

4-7、Pic_info_validate 是用于在线实时评测的配图数据集，格式和Pic_info_train一样。

 

4-8、News_pic_validate 是用于在线实时评测的标注数据集，格式和News_pic_label_train一样，只是没有标注信息。

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt  # Matlab-style plotting
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')
%matplotlib inline  
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)
from scipy import stats
from scipy.stats import norm, skew #for some statistics
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points
from subprocess import check_output
import re
import numpy as np
from PIL import Image

### 读入数据

In [2]:
train_data = pd.read_hdf('./clean_data/data.hdf5',key='news_info_train')

### 将图片按分类分出来
* final_pic_0：没有广告意图的图片名字列表；
* final_pic_1：有广告意图的图片名字列表；

In [3]:
label_0_pic = train_data.loc[train_data['label']==0]['Pics'].apply(lambda x: x.split(';'))
label_1_pic = train_data.loc[train_data['label']==1]['target_pic'].apply(lambda x: x.split(';'))
label_1_pic_all = train_data.loc[train_data['label']==1]['Pics'].apply(lambda x: x.split(';'))
label_2_pic = train_data.loc[train_data['label']==2]['Pics'].apply(lambda x: x.split(';'))
pic_all = train_data['Pics'].apply(lambda x: x.split(';'))
pic_all_list = []
for i,r in pic_all.iteritems():
    if r != ['']:
        pic_all_list.extend(r)

label_0_pic_list = []
label_1_pic_list = []
label_2_pic_list = []
label_1_pic_all_list =[]
for index in range(3):
    var_name = 'label_' + str(index) + '_pic'
    list_name = 'label_'+ str(index)+ '_pic_list'
    for i,r in globals()[var_name].iteritems():
        if r != ['']:
            globals()[list_name].extend(r)
            
for i,r in label_1_pic_all.iteritems():
    if r != ['']:
        label_1_pic_all_list.extend(r)   
pic_label_1_to_0 = list(set(label_1_pic_all_list) -set(label_1_pic_list))#[i for i in label_1_pic_all_list if i not in label_1_pic_list]
final_pic_0 = label_0_pic_list + pic_label_1_to_0
final_pic_1 = label_1_pic_list + label_2_pic_list
pic_all_list = [re.sub('.jpg','.JPG',i) for i in pic_all_list]


### 记录图片大小

In [ ]:
shape=[]
for name in pic_all_list:
    try:
        img = Image.open('./data/Pic_info_train/' + name)
        shape.append(img.size)
        img.close()
    except:
        print(name+' not in list')
    

### 计算图片的大小中位数

In [32]:
mean_x = np.median([i[0] for i in shape])
mean_y = np.median([i[1] for i in shape])